In [ ]:
import pandas as pd
import numpy as np
import json
import os
import re
from fuzzywuzzy import process

##### Tratamiento de columna category dataset Google

In [ ]:
# Para cargar todos los archivos JSON en la carpeta 

# Lista para almacenar los datos de los archivos JSON
data_list = []

# Ruta de la carpeta que contiene los archivos JSON
folder_path = './dataset/google/metadata-sitios2'

# Iterar sobre todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    # Verificar si el archivo es un archivo JSON
    if filename.endswith('.json'):
        # Construir la ruta completa del archivo
        file_path = os.path.join(folder_path, filename)
        # Leer el archivo JSON línea por línea
        with open(file_path, 'r') as f:
            for line in f:
                # Cargar cada línea como un objeto JSON y agregarlo a la lista
                data = json.loads(line)
                data_list.append(data)

# Crear el DataFrame a partir de la lista de datos
df_g_sitios = pd.DataFrame(data_list)

print(df_g_sitios.shape)
print(df_g_sitios.info())

In [ ]:
# Para generar una fila diferente por cada elemento en la lista de category
# Asegurarse de que las listas están manejadas correctamente y no hay valores NaN
df_g_sitios['category_explode'] = df_g_sitios['category'].apply(lambda x: x if isinstance(x, list) else [x] if pd.notna(x) else [])

# Expandir las listas en filas individuales
df_categories_expanded = df_g_sitios.explode('category_explode')

# Calcular el número de valores únicos
num_categorias_unicas = df_categories_expanded['category_explode'].nunique()
print(f'Número de categorías únicas: {num_categorias_unicas}')

In [ ]:
# Filtrar las filas que contienen la palabra 'restaurant' (case insensitive)
df_restaurant_categories = df_categories_expanded[df_categories_expanded['category_explode'].str.contains('restaurant', case=False, na=False)]

# Contar la frecuencia de cada categoría
print(df_restaurant_categories['category_explode'].nunique())

In [ ]:
# Convertir el array numpy en un DataFrame de pandas
unique_values = pd.DataFrame(df_restaurant_categories['category_explode'].unique())

# Guardar el DataFrame en un archivo CSV
unique_values.to_csv('valores_unicos_categories_google.csv', index=False, header=False)

##### Tratamiento de columna category dataset YELP

In [ ]:
# Cargar el archivo pickle en un DataFrame
dfy_business = pd.read_pickle("./dataset/yelp/business.pkl")

print(dfy_business.shape)

print(dfy_business.info())

In [ ]:
# Eliminar las columnas duplicadas seleccionando solo las primeras 14 columnas
dfy_business = dfy_business.iloc[:, :14]

In [ ]:
# Para generar una fila diferente por cada elemento en la lista de category
# Asegurarse de que las listas están manejadas correctamente y no hay valores NaN
dfy_business['categories_explode'] = dfy_business['categories'].apply(lambda x: x if isinstance(x, list) else [x] if pd.notna(x) else [])

# Expandir las listas en filas individuales
df_categories_expanded = dfy_business.explode('categories_explode')

# Calcular el número de valores únicos
num_categorias_unicas = df_categories_expanded['categories_explode'].nunique()
print(f'Número de categorías únicas: {num_categorias_unicas}')

In [ ]:
# Filtrar las filas que contienen la palabra 'restaurant' (case insensitive)
dfy_restaurant_categories = dfy_business[dfy_business['categories'].str.contains('restaurant', case=False, na=False)]

# Contar la frecuencia de cada categoría
print(dfy_restaurant_categories['categories'].nunique())

In [ ]:
# Leer el archivo CSV con nombres estándar de categorias provenientes de google
df_standard_categories = pd.read_csv('valores_unicos_categories_google.csv')

standard_categories = df_standard_categories['categories_standard'].tolist()

In [ ]:
# Para quitar la palabra restaurants de la columna categories para mejorar la clasificacion
# Definir una función para limpiar la categoría
def limpiar_categoria(categoria):
    if categoria.strip().lower() == 'restaurants':
        return categoria
    # Eliminar todas las apariciones de la palabra 'restaurants'
    cleaned_category = re.sub(r'\brestaurants?\b', '', categoria, flags=re.IGNORECASE).strip()
    # Eliminar comas innecesarias y espacios adicionales
    cleaned_category = re.sub(r'\s*,\s*', ', ', cleaned_category)
    cleaned_category = re.sub(r',\s*$', '', cleaned_category)
    return cleaned_category

# Aplicar la función a la columna 'categories' para crear una nueva columna 'categories_clean'
dfy_restaurant_categories['categories_clean'] = dfy_restaurant_categories['categories'].apply(limpiar_categoria)

# Mostrar algunos resultados para verificar
print(dfy_restaurant_categories[['categories', 'categories_clean']].head(1))


In [ ]:
#Para quitar la palabra 'food' de la columna categories para mejorar la clasificacion
# Definir una función para limpiar la categoría
def limpiar_categoria(categoria):
    if categoria.strip().lower() == 'food':
        return categoria
    # Eliminar todas las apariciones de la palabra 'food'
    cleaned_category = re.sub(r'\bfood?\b', '', categoria, flags=re.IGNORECASE).strip()
    # Eliminar comas innecesarias y espacios adicionales
    cleaned_category = re.sub(r'\s*,\s*', ', ', cleaned_category)
    cleaned_category = re.sub(r',\s*$', '', cleaned_category)
    return cleaned_category

# Aplicar la función a la columna 'categories' para crear una nueva columna 'categories_clean'
dfy_restaurant_categories['categories_clean'] = dfy_restaurant_categories['categories_clean'].apply(limpiar_categoria)

# Mostrar algunos resultados para verificar
print(dfy_restaurant_categories[['categories', 'categories_clean']].head(1))

In [ ]:
# Definimos una función para encontrar la categoría estándar más similar:
def find_closest_category(category, standard_categories):
    # Encontrar la categoría estándar más similar
    closest_match, score = process.extractOne(category, standard_categories)
    return closest_match, score

# Aplicar la función al DataFrame df_restaurant_categories:
dfy_restaurant_categories[['categories_standard', 'score']] = dfy_restaurant_categories['categories_clean'].apply(lambda x: pd.Series(find_closest_category(x, standard_categories)))

In [ ]:
print(dfy_restaurant_categories['categories_standard'].nunique())

In [ ]:
# Guarda el dataframe en disco en un archivo con formato parquet
dfy_restaurant_categories.to_parquet('dfy_restaurant_categories.parquet')